In [1]:
import sqlite3
from datetime import datetime

conn = sqlite3.connect('user_login_db.sqlite')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS user_login (
        id INTEGER PRIMARY KEY,
        user_id INTEGER,
        login_time TEXT,
        timestamp DATETIME,
        success INTEGER
    )
''')

conn.commit()
conn.close()

In [2]:
import sqlite3
import random
import time

conn = sqlite3.connect('user_login_db.sqlite')
cursor = conn.cursor()

user_ids = [1, 2, 3, 4, 5]

for _ in range(50):
    user_id = random.choice(user_ids)
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
    login_time = time.strftime('%H:%M:%S', time.gmtime(random.randint(0, 86399)))
    success = random.randint(0, 1)
    
    cursor.execute('''
        INSERT INTO user_login (user_id, timestamp,login_time, success)
        VALUES (?, ?, ?, ?)
    ''', (user_id, timestamp,login_time, success))
cursor.execute('SELECT * FROM user_login')
login_data = cursor.fetchall()

failed_login_counts = {}

for row in login_data:
    user_id = row[1]
    success = row[4]

    if success == 0:
        if user_id in failed_login_counts:
            failed_login_counts[user_id] += 1
        else:
            failed_login_counts[user_id] = 1

    print(f"ID: {row[0]}, User ID: {user_id}, login_time: {row[2]}, Success: {'Yes' if success == 1 else 'No'}")


conn.commit()
conn.close()

ID: 1, User ID: 3, login_time: 09:13:06, Success: No
ID: 2, User ID: 5, login_time: 21:04:33, Success: Yes
ID: 3, User ID: 3, login_time: 20:22:12, Success: No
ID: 4, User ID: 3, login_time: 11:45:35, Success: No
ID: 5, User ID: 2, login_time: 04:03:25, Success: No
ID: 6, User ID: 4, login_time: 17:44:58, Success: No
ID: 7, User ID: 5, login_time: 10:45:52, Success: No
ID: 8, User ID: 2, login_time: 10:47:24, Success: No
ID: 9, User ID: 3, login_time: 17:38:42, Success: Yes
ID: 10, User ID: 3, login_time: 22:37:11, Success: No
ID: 11, User ID: 3, login_time: 22:15:07, Success: Yes
ID: 12, User ID: 2, login_time: 16:34:09, Success: No
ID: 13, User ID: 5, login_time: 17:12:26, Success: Yes
ID: 14, User ID: 4, login_time: 15:32:19, Success: Yes
ID: 15, User ID: 3, login_time: 20:30:59, Success: Yes
ID: 16, User ID: 3, login_time: 20:38:48, Success: Yes
ID: 17, User ID: 4, login_time: 17:46:46, Success: No
ID: 18, User ID: 1, login_time: 14:39:31, Success: Yes
ID: 19, User ID: 4, login_tim

In [3]:
import sqlite3

conn = sqlite3.connect('user_login_db.sqlite')
cursor = conn.cursor()

cursor.execute('SELECT user_id, COUNT(*) FROM user_login WHERE success = 0 GROUP BY user_id')
failed_login_counts = dict(cursor.fetchall())

threshold = 3  # Adjust the threshold as needed

for user_id, count in failed_login_counts.items():
    if count > threshold:
        print(f"Anomaly detected for user {user_id}. Failed login count: {count}")
        
 


Anomaly detected for user 2. Failed login count: 8
Anomaly detected for user 3. Failed login count: 8
Anomaly detected for user 4. Failed login count: 6
Anomaly detected for user 5. Failed login count: 4


In [4]:
# Detect anomalies based on time differences between login times
threshold_seconds = 600  # Adjust the threshold in seconds (e.g., 600 seconds = 10 minutes)

conn = sqlite3.connect('user_login_db.sqlite')
cursor = conn.cursor()

cursor.execute('SELECT user_id, login_time FROM user_login ORDER BY user_id, login_time')

anomalies = []

previous_user_id = None
previous_login_time = None
for user_id, login_time in cursor.fetchall():
    if previous_user_id is not None and user_id == previous_user_id:
        time_diff = (datetime.strptime(login_time, '%H:%M:%S') - datetime.strptime(previous_login_time, '%H:%M:%S')).seconds
        if time_diff < threshold_seconds:
            anomalies.append((user_id, login_time, previous_login_time))
    previous_user_id = user_id
    previous_login_time = login_time

conn.close()

print("Anomalies (Time Differences Between Login Times):")
for anomaly in anomalies:
    user_id, login_time, previous_login_time = anomaly
    print(f"User ID: {user_id}, Current Login Time: {login_time}, Previous Login Time: {previous_login_time}")


Anomalies (Time Differences Between Login Times):
User ID: 3, Current Login Time: 09:13:06, Previous Login Time: 09:06:19
User ID: 3, Current Login Time: 20:30:59, Previous Login Time: 20:22:12
User ID: 3, Current Login Time: 20:38:48, Previous Login Time: 20:30:59
User ID: 3, Current Login Time: 22:37:44, Previous Login Time: 22:37:11
User ID: 4, Current Login Time: 06:31:40, Previous Login Time: 06:23:09
User ID: 4, Current Login Time: 17:46:46, Previous Login Time: 17:44:58
